In [1]:
import pandas as pd    
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# 忽略警告
warnings.filterwarnings('ignore') 
# 使用 ggplot 画图风格
plt.style.use('ggplot')
%matplotlib inline

In [3]:
data = pd.read_csv('./house-prices-advanced-regression-techniques/train.csv', index_col=0)

numeric=[f for f in data.drop(['SalePrice'],axis=1).columns 
         if data.drop(['SalePrice'],axis=1).dtypes[f]!='object']
# 统计类别类型（category）特征名，保存为列表
category=[f for f in data.drop(['SalePrice'],axis=1).columns 
          if data.drop(['SalePrice'],axis=1).dtypes[f]=='object']
# 输出数值类型(numerical)特征个数，类型（category）特征个数
print("numeric: {}, category: {}" .format (len(numeric),len(category)))

numeric: 36, category: 43


In [4]:
# 统计每个特征的缺失数
missing = data.isnull().sum()
# 将缺失数从大到小排序
missing.sort_values(inplace=True,ascending=False)
# 选取缺失数大于0的特征
missing = missing[missing > 0]
# 保存含缺失值特征的类型
types = data[missing.index].dtypes
# 计算缺失值百分比
percent = missing / data.shape[0]
# 将缺失值信息整合
missing_data = pd.concat([missing, percent,types],axis=1,
                         keys=['Total', 'Percent','Types'])
# 输出缺失值信息
missing_data

,Total,Percent,Types
PoolQC,1453,0.995205,object
MiscFeature,1406,0.963014,object
Alley,1369,0.937671,object
Fence,1179,0.807534,object
FireplaceQu,690,0.472603,object
LotFrontage,259,0.177397,float64
GarageYrBlt,81,0.055479,float64
GarageCond,81,0.055479,object
GarageType,81,0.055479,object
GarageFinish,81,0.055479,object


In [5]:
# 对缺失值超过15%的特征进行删除
data.drop(missing_data[missing_data['Percent'] > 0.15].
          index, axis=1, inplace=True)
# 输出数据维度
print(data.shape)

(1460, 74)


In [6]:
data['Electrical'].fillna(data['Electrical'].mode()[0], inplace=True)
cols1 = ['GarageFinish', 'GarageQual', 'GarageType', 'GarageCond', 
         'BsmtFinType2', 'BsmtExposure',
         'BsmtFinType1', 'BsmtQual', 'BsmtCond', 'MasVnrType']
# 依次便利cols1中的特征，对应缺失值用‘None’填充
for col in cols1:
    data[col].fillna('None', inplace=True)

In [7]:
data['MasVnrArea'].fillna(0, inplace=True)
year_map = pd.concat(pd.Series('YearGroup' + str(i+1), 
           index=range(1871+i*20,1891+i*20)) for i in range(0, 7))
data['GarageYrBlt'] = data['GarageYrBlt'].map(year_map)
# 对时间缺失值用‘None’填充
data['GarageYrBlt']= data['GarageYrBlt'].fillna('None')